# Day 10 
## Part 1

The question is a bit confusing but I think I just need to sort, get the differences between values, and count the 1s and 3.

In [10]:
def parse_data(s):
    return [int(x) for x in s.splitlines()]

def diffs(data):
    sorted_data = sorted(data)
    joltages = [0] + sorted_data +[sorted_data[-1] + 3]
    return[(y - x) for x, y in zip(joltages[:-1], joltages[1:])]

def part_1(data):
    ds = diffs(data)
    return ds.count(1) * ds.count(3)

test_data = parse_data('''16
10
15
5
1
11
7
19
6
12
4''')

assert part_1(test_data) == 35

In [11]:
part_1(test_data) 

35

In [13]:
test_data_2 = parse_data('''28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3''')

assert part_1(test_data_2) == 220

In [15]:
data = parse_data(open('input').read().strip())
part_1(data)

2376

## Part 2
Draw a directed graph with nodes as adapters and edges linking to the next compatible adapters. Depth first search and count the number of times we get to the end.

In [23]:
def part_2(data):
    end = max(data) + 3
    joltages = data + [0, end]
    graph = {
        adapter: [j for j in joltages if 1 <= j - adapter <= 3]
        for adapter in joltages
    }
    result = 0
    
    # DFS
    search = [0]
    while search:
        adapter = search.pop()
        if adapter == end:
            result += 1
        else:
            search.extend(graph[adapter])
            
    return result

assert part_2(test_data) == 8

In [24]:
assert part_2(test_data_2) == 19208

`part_2(data)` is taking too long. 

Some adapters will have to be in every sequence, i.e. the ones where the minimum diff in joltages to the adapter plus the maximum diff in joltages from the adapter is more than three. Calculate the number of combinations between these and multiple them together.

So partition the list of joltages into lists where each end has to be in the sequence, then run a DFS on each partition to get the number of combinations for that partition, and finally multiply the lot together.

In [44]:
def partition_joltages(joltages):
    joltages = sorted(joltages)
    last_partition = 0
    result = []
    for i, joltage in enumerate(joltages[1:-1]):
        # Will this adapter always be in the path?
        if (joltage - joltages[i - 1]) + (joltages[i + 1] - joltage) > 3:
            result.append(joltages[last_partition:i + 1])
            last_partition = i
    return(result + [joltages[last_partition:]])


In [56]:
import math

def dfs(nodes):
    start = min(nodes)
    end = max(nodes)
    graph = {
        node: [n for n in nodes if 1 <= n - node <= 3]
        for node in nodes
    }
    
    result = 0
    search = [start]
    while search:
        node = search.pop()
        if node == end:
            result += 1
        else:
            search.extend(graph[node])
            
    return result    
    

def part_2(data):
    partitions = partition_joltages(data + [0, max(data) + 3])
    
    return math.prod(dfs(p) for p in partitions)

In [57]:
assert part_2(test_data) == 8

In [58]:
assert part_2(test_data_2) == 19208

In [59]:
part_2(data)

129586085429248

This could be optimised further. Each partition is effectively a list of deltas, and the results could be at least cached or even potentially calculated without searching in a way I haven't thought of.